In [1]:
import configparser, time, re
config = configparser.ConfigParser()
config.read('hometemp.ini')
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from datetime import datetime, timedelta
from persist.database import SensorDataHandler

In [2]:
auth = config["db"]
handler = SensorDataHandler(auth['db_port'], auth['db_host'], auth['db_user'], auth['db_pw'], 'sensor_data')
handler.init_db_connection(check_table=False)
df = handler.read_data_into_dataframe()
df['timestamp'] = df['timestamp'].map(lambda x : datetime.strptime(str(x).strip(), '%Y-%m-%d %H:%M:%S'))
df = df.sort_values(by='timestamp')
df = df.reset_index()
#print(str(df['timestamp'][0].month), str(df['timestamp'][0].day))

df

INFO:persistence:Connected to the database!


,index,id,timestamp,humidity,room_temp,cpu_temp
0,4,6481,2023-08-12 11:32:54,69.500000,21.000000,37.500
1,116,6482,2023-08-12 11:42:55,70.000000,21.100000,37.500
2,232,6483,2023-08-12 11:53:01,69.800000,21.100000,36.500
3,349,6484,2023-08-12 12:03:03,69.500000,21.100000,36.500
4,466,6485,2023-08-12 12:13:04,69.500000,21.100000,36.000
...,...,...,...,...,...,...
4027,4027,10508,2023-09-10 10:57:38,68.800003,20.400000,36.024
4028,4028,10509,2023-09-10 11:07:39,68.800003,20.500000,37.485
4029,4029,10510,2023-09-10 11:17:40,68.800003,20.500000,36.024
4030,4030,10511,2023-09-10 11:27:42,68.800003,20.600000,36.511


In [ ]:
# measurement data collection started on 12th august 2023, 11 hours 32 minutes 54 seconds
assert df['timestamp'][0].day == 12 and df['timestamp'][0].month == 8 and df['timestamp'][0].year == 2023 and df['timestamp'][0].hour == 11 and df['timestamp'][0].minute == 32 and df['timestamp'][0].second == 54

In [ ]:
df[["humidity", "room_temp", "cpu_temp"]].describe()

In [ ]:
corr_matrix = df[["humidity", "room_temp", "cpu_temp"]].corr()
#corr_matrix = df[["humidity", "room_temp"]].corr()
corr_matrix

In [ ]:
def draw_plots(df, show_heatmap=True):
    subplots = 3 if show_heatmap else 2
    sns.set_theme(style="darkgrid")#sns.set(style="whitegrid")
    fig = plt.figure(figsize=(25, 12))
    gs = fig.add_gridspec(2, 2, height_ratios=[2, 2])  # 2 rows, 1 column

    # Temperature Measurements
    plt.subplot(gs[0])
    sns.lineplot(label="Home", x="timestamp", y="room_temp", data=df)
    plt.title("Temperature Over Time")
    plt.xlabel("Time")
    plt.ylabel("Temp (°C)")
    plt.legend()
    plt.xticks(rotation=45)
    #plt.tight_layout()
    #plt.show()

    # Humidity Measurement
    plt.subplot(gs[1])
    sns.lineplot(x="timestamp", y="humidity" , data=df, color='purple')
    plt.title("Humidity Over Time")
    plt.xlabel("Time")
    plt.ylabel("Humidity (%)")
    plt.xticks(rotation=45, ha='right')
    plt.gca().xaxis.grid(True)
    plt.gca().set_facecolor('#f5f5f5')
    sns.despine(left=True, bottom=True)

    df_last_24h = df[df["timestamp"] >= datetime.now() - timedelta(hours=25)]

    # Temperature Measurements last 24 h
    plt.subplot(gs[2])
    sns.lineplot(label="Home", x="timestamp", y="room_temp",marker='o',markersize=6, data=df_last_24h)
    plt.title("Temperature Last 24 Hours")
    plt.xlabel("Time")
    plt.ylabel("Temp (°C)")
    plt.legend()
    plt.xticks(rotation=45)

    # Humidity Measurements last 24 h
    plt.subplot(gs[3])
    sns.lineplot(x="timestamp", y="humidity", marker='o', markersize=6, color='purple', data=df_last_24h)
    plt.title("Humidity Last 24 Hours")
    plt.xlabel("Time")
    plt.ylabel("Humidity (%)")
    plt.xticks(rotation=45, ha='right')
    plt.gca().xaxis.grid(True)
    plt.gca().set_facecolor('#f5f5f5')
    sns.despine(left=True, bottom=True)

    # Correlations
    if show_heatmap:
        plt.subplot(1, subplots, 3) 
        #mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
        #sns.heatmap(corr_matrix, annot=True,mask=mask, cmap="coolwarm", vmin=-1, vmax=1)
        sns.heatmap(df[["humidity", "room_temp", "cpu_temp"]].corr(), annot=True, cmap="coolwarm")
        plt.title("Correlation Heatmapfor All Categories")

    plt.tight_layout()
    name = datetime.now().strftime("%d-%m-%Y")
    #plt.savefig(f"plots/{name}.pdf")
    plt.show()
    plt.close()
    sns.reset_defaults()

draw_plots(df, False)

In [ ]:
plt.figure(figsize=(25, 8))
sns.lineplot(label="CPU", x="timestamp", y="cpu_temp", data=df)
plt.title("Temperature Over Time")
plt.xlabel("Time")
plt.ylabel("Temp (°C)")
plt.legend()
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

plt.figure(figsize=(25, 8))
sns.lineplot(label="CPU", x="timestamp", y="cpu_temp",marker='o',markersize=6, data=df[df["timestamp"] >= datetime.now() - timedelta(hours=25)])
plt.title("Temperature Last 24h")
plt.xlabel("Time")
plt.ylabel("Temp (°C)")
plt.legend()
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [ ]:
csv_file = 'fitness.csv'  
fitness = pd.read_csv(csv_file, parse_dates=['time'])
fitness.describe()

In [ ]:
def plot_fitness(fitness):
    plt.figure(figsize=(25, 10))

    # Visualization 1: Line plot for weight over time
    plt.subplot(2, 2, 1)
    sns.lineplot(data=fitness, x='time', y='weight')
    plt.xticks(rotation=45)
    plt.title('Weight over Time')

    # Visualization 2: Line plot for BMI over time
    plt.subplot(2, 2, 2)
    sns.lineplot(data=fitness, x='time', y='bmi')
    plt.xticks(rotation=45)
    plt.title('BMI over Time')

    # Visualization 3: Line plot for body fat over time
    plt.subplot(2, 2, 3)
    sns.lineplot(data=fitness, x='time', y='body_fat')
    plt.xticks(rotation=45)
    plt.title('Body Fat over Time')

    # Visualization 4: Line plot for muscle over time
    plt.subplot(2, 2, 4)
    sns.lineplot(data=fitness, x='time', y='muscle')
    plt.xticks(rotation=45)
    plt.title('Muscle over Time')

    plt.tight_layout()
    plt.show()

def plot_fitness_corr(fitness):
    correlation_matrix = fitness[["weight", "bmi", "body_fat", "muscle", "kcal", "visceral"]].corr()
    plt.figure(figsize=(10, 8))
    mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))
    sns.heatmap(correlation_matrix, annot=True,mask=mask, cmap='coolwarm', center=0)
    plt.title('Correlation Heatmap')
    plt.show()

#plot_fitness(fitness)
#plot_fitness_corr(fitness)